# Load Database

In [1]:
import os
import pandas as PD

In [2]:
import django
MODULE_PATH = os.path.dirname(os.getcwd())
os.chdir(os.path.dirname(MODULE_PATH))
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
django.setup()

In [3]:
import app_proj.utility as UT
print([x for x in dir(UT) if '_' not in x and len(x) > 8])

['BaseManager', 'ConvertFigureToJson', 'DeleteTable', 'GetTableCounts', 'GetTableDictionary', 'InsertBulk', 'InsertSingle']


In [4]:
import business_module.models as BM
import business_module.logic.custom as CT
DATA_PATH = os.path.join(MODULE_PATH, 'data')
print([x for x in dir(CT) if '_' not in x and len(x) > 8])

['MyFunction']


In [5]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### CRUD Code

In [6]:
infoLs = UT.GetTableCounts()
PD.DataFrame(infoLs)

,Module,Table,Count
0,business_module,LegoSet,6181


In [7]:
# get data from CSV

setsPath = os.path.join(DATA_PATH, 'brickset_set_filter.csv')
setsDf = PD.read_csv(setsPath)
setsDf[20:23]
setsDf.shape

,set_no,name,price_store,price_new,price_used,rating_value,rating_votes,theme_group,theme,subtheme,main_tag,year,volume,weight,piece_cnt,minifig_cnt
20,6627-1,Convertible,NaN,88.0,6.0,5.0,7.0,Modern day,Town,Vehicles,NaN,1980.0,NaN,NaN,37.0,1.0
21,6647-1,Highway Repair,NaN,167.0,6.0,4.0,1.0,Modern day,Town,Maintenance,NaN,1980.0,NaN,NaN,55.0,1.0
22,6648-2,Dump Truck,NaN,149.0,7.0,4.0,2.0,Modern day,Town,Construction,NaN,1980.0,NaN,NaN,42.0,1.0


(6181, 16)

In [8]:
setsDf = setsDf.rename({'set_no': 'SetNo', 'name': 'Name', 'year': 'Year',
                        'theme_group': 'ThemeGroup', 'theme': 'Theme', 'subtheme': 'Subtheme',
                        'price_store': 'PriceStore', 'price_new': 'PriceNew', 'price_used': 'PriceUsed',
                        'piece_cnt': 'PieceCount', 'minifig_cnt': 'MinifigCount'
                       }, axis='columns')
setsDf = setsDf.drop(['rating_value', 'rating_votes', 'main_tag', 'volume', 'weight'], axis=1, errors='ignore')
setsDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6181 entries, 0 to 6180
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SetNo         6181 non-null   object 
 1   Name          6181 non-null   object 
 2   PriceStore    4767 non-null   float64
 3   PriceNew      5780 non-null   float64
 4   PriceUsed     5489 non-null   float64
 5   ThemeGroup    6181 non-null   object 
 6   Theme         6181 non-null   object 
 7   Subtheme      4494 non-null   object 
 8   Year          6181 non-null   float64
 9   PieceCount    6137 non-null   float64
 10  MinifigCount  6181 non-null   float64
dtypes: float64(6), object(5)
memory usage: 531.3+ KB


In [9]:
setsLs = setsDf.to_dict('records')
print(len(setsLs))

6181


In [10]:
# insert in bulk

UT.InsertBulk('business_module', 'LegoSet', setsLs)

infoLs = UT.GetTableCounts()
PD.DataFrame(infoLs)

bulk inserted


,Module,Table,Count
0,business_module,LegoSet,6181


In [11]:
# select entire table

tableLs = UT.GetTableDictionary('business_module', 'LegoSet')
PD.DataFrame(tableLs)[20:23]

,id,SetNo,Name,Year,ThemeGroup,Theme,Subtheme,PriceStore,PriceNew,PriceUsed,PieceCount,MinifigCount
20,74195,6627-1,Convertible,1980,Modern day,Town,Vehicles,NaN,88.0,6.0,37.0,1
21,74196,6647-1,Highway Repair,1980,Modern day,Town,Maintenance,NaN,167.0,6.0,55.0,1
22,74197,6648-2,Dump Truck,1980,Modern day,Town,Construction,NaN,149.0,7.0,42.0,1


In [13]:
# select one row by parameters

paramDx = {'SetNo': '6627-1'}
rowDx = UT.GetRow('business_module', 'LegoSet', paramDx)
rowDx

{'_state': <django.db.models.base.ModelState at 0x240fcf0c910>,
 'id': 74195,
 'SetNo': '6627-1',
 'Name': 'Convertible',
 'Year': 1980,
 'ThemeGroup': 'Modern day',
 'Theme': 'Town',
 'Subtheme': 'Vehicles',
 'PriceStore': None,
 'PriceNew': 88.0,
 'PriceUsed': 6.0,
 'PieceCount': 37,
 'MinifigCount': 1}

In [14]:
# select when row doesn't exist

paramDx = {'SetNo': '6627-2'}
rowDx = UT.GetRow('business_module', 'LegoSet', paramDx)
rowDx

{}

In [15]:
# delete all rows

# UT.DeleteTable('business_module', 'LegoSet')

# infoLs = UT.GetTableCounts()
# PD.DataFrame(infoLs)